
# KnightL on Chess Board

* [KnightL on a Chessboard](https://www.hackerrank.com/contests/rookierank-2/challenges/knightl-on-chessboard/problem)

Wnen a Knight can take L shape move in which the distances it moves in x and y directions are represented as ```[dx,dy]```, as long as he is on the board, we denote the knight as ```knight[1,3]```. ```[dx,dy]``` can be any of ```[1,3],[1,-3],[-1,3],[-1,-3]```.


To move from ```(3,4)``` to ```(0,0)```, the ```knight[1,3]``` takes minimum 3 steps ```(3,4) -> (4,2) -> (2,1) -> (0,0).```

<img src="../image/KnightChess.jpg" align="left" width=300/>

# Destinations where the knight can go as a Graph

The locations where the ```knight[d0,d1]``` can go from ```(x,y)``` can be represented as a graph whose nodes are the coordiantes of the locations.

From the node ```(x,y)```, take one move from ```[1,3],[1,-3],[-1,3],[-1,-3]``` to get to the next node. Try all the moves until all the location the knight can go are exhausted.


# Setup

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.ERROR)
logger = logging.getLogger('LOGGER_NAME')

In [2]:
import threading
from itertools import (
    combinations,
    combinations_with_replacement,
    permutations,
    product,
)
import operator
import matplotlib.pyplot as plt

# Constant

In [3]:
N = 5  # Board size
ORIGINATION = (0,0)
DESTINATION = (4,4)

# All the moves the knight can make

In [4]:
(dx, dy) = (1, 3)

# All possible signs
signs = list(product([1, -1], repeat=2))

# All possible combination of [(+/-, +/-), (dx, dy)]
combinations = product(signs, [(dx, dy), (dy, dx)])

# All possible moves (x,y)
movements = [
    tuple(map(operator.mul, sign, xy))
    for (sign, xy) in combinations
]
movements

[(1, 3), (3, 1), (1, -3), (3, -1), (-1, 3), (-3, 1), (-1, -3), (-3, -1)]

In [5]:
def create_possible_movements(dx, dy):
    """Generate all possible (dx, dy) and (dy, dx) combinations without duplicates.
    (dx, dy)
    (dy, dx)
    (dx, -dy)
    (dy, -dx)
    (-dx, dy)
    (-dy, dx)
    (-dx, -dy)
    (-dy, -dx)
    
    For (dx, dy)=(2,2),
    (2,2)
    (2,-2)
    (-2,2)
    (-2,-2)
    """
    # All possible signs
    signs = list(product([1, -1], repeat=2))

    # All possible combination of [(+/-, +/-), (dx, dy)]
    combinations = product(signs, [(dx, dy), (dy, dx)])

    # All possible moves (dx,dy) and (dy, dx)
    movements = set([
        tuple(map(operator.mul, sign, dxdy))
        for (sign, dxdy) in combinations
    ])
    return movements

In [6]:
print(create_possible_movements(1,3))
print(create_possible_movements(1,1))

{(-3, -1), (3, -1), (-3, 1), (3, 1), (-1, -3), (1, -3), (-1, 3), (1, 3)}
{(-1, 1), (1, 1), (-1, -1), (1, -1)}


# Graph implementation

* [Generate a graph using Dictionary in Python](https://www.geeksforgeeks.org/generate-graph-using-dictionary-python/)
* [Implement graphs in python like a pro](https://medium.com/youstart-labs/implement-graphs-in-python-like-a-pro-63bc220b45a0)
* [How to implement a graph in Python](https://www.educative.io/edpresso/how-to-implement-a-graph-in-python)

In [7]:
from network_graph import (
    Graph
)

In [8]:
graph = Graph()
graph.add_edge((0,0), (1,2))
graph.has_edge((0,0), (1,2))

True

# Chess Board

In [9]:
size = N
ticks = list(range(0, N, 1))
ticks

[0, 1, 2, 3, 4]

In [10]:
coordinates = list(product(ticks, repeat=2))
for (x, y) in coordinates:
    print(f"(x,y)={x,y}")
    
print(f"\nTotal {len(coordinates)}")

(x,y)=(0, 0)
(x,y)=(0, 1)
(x,y)=(0, 2)
(x,y)=(0, 3)
(x,y)=(0, 4)
(x,y)=(1, 0)
(x,y)=(1, 1)
(x,y)=(1, 2)
(x,y)=(1, 3)
(x,y)=(1, 4)
(x,y)=(2, 0)
(x,y)=(2, 1)
(x,y)=(2, 2)
(x,y)=(2, 3)
(x,y)=(2, 4)
(x,y)=(3, 0)
(x,y)=(3, 1)
(x,y)=(3, 2)
(x,y)=(3, 3)
(x,y)=(3, 4)
(x,y)=(4, 0)
(x,y)=(4, 1)
(x,y)=(4, 2)
(x,y)=(4, 3)
(x,y)=(4, 4)

Total 25


In [11]:
def move(n, position, movement):
    """
    Args:
        n: size
        position: coordinate as (x,y)
        movement: distance to move as (dx, dy)
    Returns:
        new position (x, y) if it is within (0 <= x|y < n-1), othewise None
    """
    x = position[0]
    y = position[1]
    dx = movement[0]
    dy = movement[1]
    new_x = x + dx
    new_y = y + dy
    if (0 <= new_x < n) and (0 <= new_y < n):
        return (new_x, new_y)
    else: 
        # logger.debug("move(): cannot go to %s." % ((new_x, new_y),))
        return None

Can go to (4, 4)
Can go to (2, 0)
Can go to (4, 2)
Can go to (0, 0)


In [ ]:
position = (1,3)
for movement in movements:
    destination = move(5, position, movement)
    if destination:
        print(f"Can go to {destination}")


### Check if the graph has (current -> destination) path


In [12]:
Lock = threading.Lock()


def check_path_and_update_graph(current, destination, graph, path):
    """
    A node in the graph can be arrived via different routes. If the destination is not yet 
    visited via the (current->destination) path, add the destination and the directed path
    to the graph. To avoid circlyc route, add (destination->current) as well.
    
    The objective of the entire program is to find the shortest path, not to find the
    unique route. If we already went through (src->dst), going back (dst->src) only
    lead to the longer route. Hence prevent going back the path already taken.
    
    However, route 1 -> 4 -> 2 -> 3 may have been taken already but the shorter route is
    1 -> 2 -> 3 to the target destination 3. Only checking if the edge/path is taken will
    prevent finding the shortest path. Hence if the route up to the current destination
    is shorter, then return True to allow exploring shorter route.
            
    Args:
        current: current position
        destination: destination coordinate
        graph: graph of destinations
        
    Returns:
        True if new path, otherwise False
    """
    visit_further = False
    with Lock:
        # --------------------------------------------------------------------------------
        # Check if graph alreadyhas has (current -> destination) path with Graph.has_edge().
        # --------------------------------------------------------------------------------
        if not graph.has_edge(current, destination):
            logger.debug(
                "check_path_and_update_graph(): %s is not yet visited from %s. Adding to the graph..." 
                % (destination, current)
            )
            graph.add_node(current)
            graph.add_node(destination)
            graph.add_edge(current, destination)
            graph.add_edge(destination, current)
            graph.set_min_path_length_to_node(destination, path)

            logger.debug("check_path_and_update_graph(): updated graph: [%s]" % graph.get_nodes())
            return True
        if not graph.has_edge(destination, current):
            logger.debug(
                "check_path_and_update_graph(): %s is not yet visited from %s. Adding to the graph..." 
                % (destination, current)
            )
            graph.add_node(current)
            graph.add_node(destination)
            graph.add_edge(current, destination)
            graph.add_edge(destination, current)
            graph.set_min_path_length_to_node(current, path[:len(path)-1])

            logger.debug("check_path_and_update_graph(): updated graph: [%s]" % graph.get_nodes())
            return True        
        else:
            assert graph.get_min_path_length_to_node(destination) > -1
            if graph.get_min_path_length_to_node(destination) >= len(path):
                graph.set_min_path_length_to_node(destination, path)
                return True
            else:
                return False

In [14]:
def visit(n, position, movements, graph, steps, target, path):
    """Visit all the locations reachable from the position and add the locations and directed
    paths to there in the graph.
    
    Args:
        n: size of the board
        position: current position
        movement: avialable move to the next destinations
        graph: graph object to update
        steps: current steps taken by the knight
        target: target destination coordinate

    Returns: steps taken to the destination
    
    """
    logger.debug("visit(): position: %s movements: %s steps: %s." % (position, movements, steps))
    results = []
    
    # --------------------------------------------------------------------------------
    # If already at the target, explore furhter only increase the steps.
    # Hence it is already the potential shortest path found.
    # No further exploration required. Resturn.
    # --------------------------------------------------------------------------------
    if position == target:
        results.extend([steps])
        return results
        
        
    for movement in movements:
        destination = move(n, position, movement)

        logger.debug(
            "visit(): position %s steps %s movement is %s destinatm %s\npath %s" % 
            (position, steps, movement, destination, path)
        )
        
        if destination is not None:
            # --------------------------------------------------------------------------------
            # If the destination is not yet visited from the , add it to the graph and explore further
            # from the destination
            # --------------------------------------------------------------------------------
            if check_path_and_update_graph(position, destination, graph, path + [destination]):
                logger.debug("Moving to the destination %s." % (destination,))
                found = visit(n, destination, movements, graph, steps+1, target, path + [destination])
                results += found
                
    return results

## Build a directed graph

* [DiGraph—Directed graphs with self loops](https://networkx.org/documentation/stable/reference/classes/digraph.html)


# Shortest Path 


In [17]:
def get_shortest_paths(graph, n, dx, dy, origination, destination):
    """Get the shortest paths from origination to destination
    """
    movements = create_possible_movements(dx, dy)
    steps_to_target = visit(
        n=n, 
        position=origination, 
        movements=movements, 
        graph=graph, 
        steps=0, 
        target=destination, 
        path=[origination]
    )
    if len(steps_to_target) > 0:
        return min(steps_to_target)
    else:
        return -1

In [18]:
graph = Graph()
graph.add_node(ORIGINATION)
get_shortest_paths(graph=graph, n=N, dx=1, dy=1, origination=ORIGINATION, destination=DESTINATION)

4

---

# [KnightL on a Chessboard](https://www.hackerrank.com/contests/rookierank-2/challenges/knightl-on-chessboard/problem)

In [19]:
def get_shortest_route_sizes(n):
    """
    For all the possible move=(dx, dy) where move is the movement an object can make 
    in x and y directions, find the size of the shortest route from the origination 
    (0,0) to the destination (n-1,n-1) for each (dx, dy).
    
    If there is no such route, then the size is -1.
    
    Create a 2D matrix where each row corresponds with dx where dx: 0 <= dx < n and
    each column corresponds with dy where dy: 0 <= dy < n.
    
    Args:
        n: Board size
    Returns:
        2D matrix of the shortest route size    
    """
    origination = (0,0)
    destination = (n-1, n-1)

    result = []
    for dx in range(1, n):
        shortests = []
        for dy in range(1, n):
            graph = Graph()
            graph.add_node(origination)
            steps = get_shortest_paths(
                graph=graph, n=n, dx=dx, dy=dy, origination=origination, destination=destination
            )
            shortests.append(steps)
            
        result.append(shortests)
        
    return result

In [20]:
get_shortest_route_sizes(n=5)

[[4, 4, 2, 8], [4, 2, 4, 4], [2, 4, -1, -1], [8, 4, -1, 1]]